# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
cities_df = pd.read_csv("./WeatherPy_output/weather_data.csv")
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,21.72,47,0,1.70,IN,1639890609
1,1,rikitea,-23.1203,-134.9692,25.28,74,100,5.51,PF,1639890609
2,2,itoman,26.1247,127.6694,19.21,52,40,5.66,JP,1639890610
3,3,atuona,-9.8000,-139.0333,25.69,75,32,6.53,PF,1639890610
4,4,qaanaaq,77.4840,-69.3632,-12.61,75,100,3.68,GL,1639890610
...,...,...,...,...,...,...,...,...,...,...
554,554,vega de alatorre,20.0333,-96.6333,23.72,88,5,2.05,MX,1639891470
555,555,lima,-12.0432,-77.0282,18.83,79,40,5.66,PE,1639891471
556,556,del rio,29.3627,-100.8968,12.46,49,90,6.17,US,1639891471
557,557,sibut,5.7180,19.0739,24.52,35,99,1.65,CF,1639891471


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# convert humidity values to float and set the weights
weights = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
#Ideal Conditions
    #A max temperature lower than 80 degrees but higher than 70.
        #As all temperatures were pulled in Celcius, these values become 26.6667 and 21.1111 respectively
    #Wind speed less than 10 mph.
    #Zero cloudiness.

ideal_weather_df = cities_df[(cities_df["Max Temp"]>21.1111)&
                             (cities_df["Max Temp"]<26.6667)&
                             (cities_df["Wind Speed"]<10)&
                             (cities_df["Cloudiness"]==0)]


ideal_weather_df



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,21.72,47,0,1.70,IN,1639890609
106,106,formosa,-26.1775,-58.1781,24.99,83,0,2.06,AR,1639890757
111,111,gualeguaychu,-33.0094,-58.5172,22.24,72,0,5.42,AR,1639890759
169,169,moron,-34.6534,-58.6198,23.87,66,0,2.24,AR,1639890834
222,222,anjangaon,21.1631,77.3094,21.21,39,0,0.68,IN,1639890967
253,253,partur,19.6000,76.2167,21.68,48,0,2.84,IN,1639890975
329,329,tigre,-34.4260,-58.5796,23.49,66,0,4.63,AR,1639891112


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# find the closest hotel to each of the above locations

hotel_name = []
city_name = []
country_name = []

for index, row in ideal_weather_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "keyword": "hotel",
        "location": f"{ideal_weather_df.loc[index,'Lat']},{ideal_weather_df.loc[index,'Lng']}",
        "radius": 5000,
        "key": g_key,
    }

    response = requests.get(base_url, params=params).json()

    hotel_name.append(response['results'][0]['name'])


hotel_df=ideal_weather_df

hotel_df["Hotel Name"] = hotel_name

hotel_df = hotel_df[['Hotel Name','City','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']]

hotel_df

<ipython-input-64-6ffcbced940b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_name


,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hotel Polo Orchid,tura,25.5198,90.2201,21.72,47,0,1.70,IN,1639890609
106,Howard Johnson by Wyndham Formosa Casino,formosa,-26.1775,-58.1781,24.99,83,0,2.06,AR,1639890757
111,Hotel Aguaý,gualeguaychu,-33.0094,-58.5172,22.24,72,0,5.42,AR,1639890759
169,Hotel Haedo,moron,-34.6534,-58.6198,23.87,66,0,2.24,AR,1639890834
222,OYO 76276 Hotel Vrindavan,anjangaon,21.1631,77.3094,21.21,39,0,0.68,IN,1639890967
253,Shree Sai Hotel,partur,19.6000,76.2167,21.68,48,0,2.84,IN,1639890975
329,Hotel Villa Victoria de Tigre,tigre,-34.4260,-58.5796,23.49,66,0,4.63,AR,1639891112


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))